### 1 - Import test data

In [20]:
import csv
from datetime import datetime

input_file = "Bitbay_BTCUSD_1h.csv"
output_file = "BTCUSD_converted.csv"

with open(input_file, newline='') as infile, open(output_file, mode='w', newline='') as outfile:
    # Pomijamy pierwszą linię (Timezones are UTC,...)
    next(infile)
    
    reader = csv.DictReader(infile)
    writer = csv.writer(outfile)
    
    # Zapis nagłówka
    writer.writerow(["Gmt time", "Open", "High", "Low", "Close", "Volume"])
    
    for row in reader:
        try:
            # Niektóre pliki mają spacje w nazwach kolumn, upewnij się, że nazwy są poprawne
            ts = float(row["Unix Timestamp"])
            dt = datetime.utcfromtimestamp(ts).strftime("%d.%m.%Y %H:%M:%S.%f")[:-3]
            
            open_ = row["Open"]
            high = row["High"]
            low = row["Low"]
            close = row["Close"]
            volume = row["Volume BTC"]  # lub Volume USD jeśli wolisz
            
            writer.writerow([dt, open_, high, low, close, volume])
        except Exception as e:
            print(f"Nie udało się przetworzyć wiersza: {row}, błąd: {e}")

print(f"Konwersja zakończona. Zapisano do {output_file}")


/tmp/ipykernel_26124/3202627635.py:21: DeprecationWarning:

datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).



Konwersja zakończona. Zapisano do BTCUSD_converted.csv


In [21]:
%pip install ta


Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import ta

df = pd.read_csv("BTCUSD_converted.csv")
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
df=df[df.High!=df.Low]
df.set_index("Gmt time", inplace=True)

In [23]:


from ta.trend import EMAIndicator

df["EMA_slow"] = EMAIndicator(
    close=df["Close"],
    window=50
).ema_indicator()

df["EMA_fast"] = EMAIndicator(
    close=df["Close"],
    window=30
).ema_indicator()

from ta.momentum import RSIIndicator

df["RSI"] = RSIIndicator(
    close=df["Close"],
    window=10
).rsi()

from ta.volatility import BollingerBands

bb = BollingerBands(
    close=df["Close"],
    window=15,
    window_dev=1.5
)

df["BB_upper"] = bb.bollinger_hband()
df["BB_middle"] = bb.bollinger_mavg()
df["BB_lower"] = bb.bollinger_lband()

from ta.volatility import AverageTrueRange

atr = AverageTrueRange(
    high=df["High"],
    low=df["Low"],
    close=df["Close"],
    window=7
)

df["ATR"] = atr.average_true_range()

df

,Open,High,Low,Close,Volume,EMA_slow,EMA_fast,RSI,BB_upper,BB_middle,BB_lower,ATR
Gmt time,,,,,,,,,,,,
2020-10-12 11:00:00,11213.57,11213.57,11213.00,11213.00,0.059030,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2020-10-12 10:00:00,11305.14,11305.14,11213.57,11213.57,0.387900,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2020-10-12 09:00:00,11390.00,11399.99,11305.14,11305.14,0.014090,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2020-10-12 08:00:00,11361.31,11390.00,11361.31,11390.00,0.001756,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2020-10-12 05:00:00,11300.05,11361.31,11300.05,11361.31,0.000498,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-11-05 15:00:00,7399.00,7414.00,7399.00,7413.61,4.950000,7216.917327,7239.851519,57.083180,7446.209495,7289.003333,7131.797172,173.896533
2017-11-05 12:00:00,7378.33,7399.00,7378.33,7399.00,0.000418,7224.057824,7250.119163,56.294866,7460.426229,7298.940667,7137.455105,154.094171
2017-11-05 11:00:00,7265.02,7399.00,7265.02,7265.02,0.209200,7225.664184,7251.080507,49.350536,7456.995543,7295.275333,7133.555124,151.220718


In [ ]:
backcandles_number = 4
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0


df=df[-10000:-1]
from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, backcandles_number) , axis=1) #if row.name >= 20 else 0

 49%|████▉     | 4937/9999 [00:03<00:03, 1417.00it/s]

In [ ]:
backcandles_number = 4
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BB_lower'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BB_upper'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0
print(backcandles_number)
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, backcandles_number), axis=1)

4


100%|██████████| 9998/9998 [00:18<00:00, 538.35it/s]


In [ ]:
df[df.TotalSignal != 0].head(20)
print(df.size)

149985


In [ ]:
import numpy as np
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

/tmp/ipykernel_26124/2345118794.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)


In [ ]:
%pip install plotly

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['BB_lower'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BB_upper'], 
                           line=dict(color='green', width=1), 
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'], 
                           line=dict(color='black', width=1), 
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'], 
                           line=dict(color='blue', width=1), 
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

Note: you may need to restart the kernel to use updated packages.


In [ ]:
def SIGNAL():
    return df.TotalSignal

In [ ]:
%pip install backtesting

from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 3000
    slcoef = 1.1
    TPSLRatio = 1.5
    rsi_length = 16
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrat, cash=250, margin=1/30)

Note: you may need to restart the kernel to use updated packages.


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

/usr/local/python/3.12.1/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning:

Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.



Loading BokehJS ...

/tmp/ipykernel_26124/898799107.py:38: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported by this class. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (see e.g. class `backtesting.lib.FractionalBacktest`.

/tmp/ipykernel_26124/898799107.py:38: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [ ]:
bt.run()

Start                                     0.0
End                                    9998.0
Duration                               9998.0
Exposure Time [%]                         0.0
Equity Final [$]                        250.0
Equity Peak [$]                         250.0
Return [%]                                0.0
Buy & Hold Return [%]               -29.11876
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Alpha [%]                                 0.0
Beta                                      0.0
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                    

In [ ]:
bt.plot()

GridPlot(id='p1191', ...)